In [8]:
import os, sys
import numpy as np
import pandas as pd

In [4]:
spellEnd=28
simEnd=28
weather= "wet"

In [69]:
def getmobs(spellEnd, simEnd, weather):
    data_file_delimiter=","
    workdir = "../results/uqrMaster/"
    directory = workdir +"6to"+str(spellEnd)+"to"+str(simEnd)+str(weather)+"/"
    name_file = directory+"len_orgs.txt"
    largest_column_count = 0
    with open(name_file, 'r') as temp_f:
        lines = temp_f.readlines()

        for l in lines:
            # Count the column count for the current line
            #print(l)
            column_count = len(l.split(data_file_delimiter)) + 1
            # Set the new most column count
            largest_column_count = column_count if largest_column_count < column_count else largest_column_count
    column_names = [i for i in range(0, column_count)]
    lenOrg = pd.read_csv(name_file, header=None, delimiter=data_file_delimiter, names=column_names)
    name_file = directory+"st_orgs.txt"
    largest_column_count = 0
    with open(name_file, 'r') as temp_f:
        lines = temp_f.readlines()

        for l in lines:
            # Count the column count for the current line
            #print(l)
            column_count = len(l.split(data_file_delimiter)) + 1
            # Set the new most column count
            largest_column_count = column_count if largest_column_count < column_count else largest_column_count
    print(largest_column_count)
    stOrg = pd.read_csv(directory+"st_orgs.txt", header=None,lineterminator='\n', delimiter=data_file_delimiter, names=column_names)
    otOrg = pd.read_csv(directory+"ot_orgs.txt", header=None,lineterminator='\n', delimiter=data_file_delimiter, names=column_names)
    time = pd.read_csv(directory+"time.txt", header=None, lineterminator='\n',delimiter=data_file_delimiter)
    lenOrg['time']=time
    stOrg['time']=time
    otOrg['time']=time

    mot = pd.melt(otOrg, id_vars="time")
    mst = pd.melt(stOrg, id_vars="time")
    mobs = pd.melt(lenOrg, id_vars="time")
    mobs['st'] = mst['value']
    mobs['ot'] = mot['value']

    mobs = mobs.dropna()
    mobs = mobs.iloc[np.where(mobs['ot']==2)[0],]
    mobs.loc[mobs['st']==4, 'st'] = 1
    mobs = mobs.rename(columns={"value":"length"})
    if weather == "wet":
        mobs['scenario'] = "wetter&colder"
    else:
        mobs['scenario'] = "drier&warmer"
    if spellEnd == 28:
        mobs['growth'] =  "b) 21-28d"
    else:
        mobs['growth'] = "a) 9-16d"
        
    return mobs

In [70]:

#suc7d = getmobs(16,28,"wet")
suc7d = getmobs(16,28,"dry")
suc21W = getmobs(28,28,"wet")
suc21d = getmobs(28,28,"dry")


1739


ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [ ]:

suc7W = getmobs("6to16wet")
suc7d = getmobs("6to16dry")
suc21W = getmobs("6to28wet")
suc21d = getmobs("6to28dry")


suc7W$scenario = "wetter&colder"
suc21W$scenario = "wetter&colder"
suc7d$scenario = "drier&warmer"
suc21d$scenario = "drier&warmer"

mall7 = rbind(suc7W,suc7d)
mall7$growth = "a) 9-16d"
mall21 = rbind(suc21W,suc21d)
mall21$growth = "b) 21-28d"

mall = rbind(mall7,mall21)

malla = aggregate(mall$length,
                  FUN = sum,
                  by=list("time" = mall$time,
                          "ot" = mall$ot,
                          "st" = mall$st,
                          "scenario" = mall$scenario,
                          "growth" = mall$growth))

malla$st = as.factor(malla$st)
malla$organTypes = ifelse(malla$ot==2,
                          ifelse(malla$st==1,"root0",
                                 ifelse(malla$st==2 ,"root1","root2")),
                          ifelse(malla$ot == 3, "stem",
                                 "leaf"))
malla$organTypes =factor(malla$organTypes, 
                         levels = c("root0","root1","root2","stem","leaf"))
p1=ggplot(data=malla[malla$growth==unique(malla$growth)[1],], 
          aes(x=time, y=x,
              col =organTypes,size=scenario,alpha=scenario))+
  
  geom_line()+
  scale_size_manual(breaks=c("drier&warmer","wetter&colder"),name="",
                    values = c(1.5,0.5), drop = FALSE)+ 
  scale_alpha_manual(breaks=c("drier&warmer","wetter&colder"),name="",
                     values = c(0.3,1), drop = FALSE)+ 
  xlab(NULL)+
  facet_grid(ot ~ growth, scales = "free_y")+
  #scale_color_brewer(palette="Set2")+
  # scale_color_manual(breaks=c("2-1", "3-1", "4-1" ,
  #                             "2-2", "2-3" ,"2-4" ,"3-4"),name="",
  #                   labels=c("r1","stem","leaf"),
  #                   values = c("#8DA0CB","#FC8D62","#66C2A5"))+  
  labs(colour = NULL, linetype =NULL)+
  scale_color_manual(name=NULL,
                     #breaks=c(2,3,4),
                     #labels=c("root","stem","leaf"),
                     values = c("#8DA0CB","#e78ac3","#a6d854","#FC8D62","#66C2A5"))+  
  ylab(NULL)+ 
  #scale_y_continuous(sec.axis = sec_axis(~.*10-7.6, name = "cumulative transpiration"))+
  theme(legend.text = element_text( size=15),
        legend.title = element_blank(),
        legend.position="bottom",#c(0.5,0.1),
        legend.direction = "horizontal",
        #legend.box = "vertical",
        panel.grid.minor = element_line(colour="white"),
        panel.grid.major = element_line(colour="white"),
        panel.background = element_rect(fill="white", color="black"),
        strip.text.y =element_blank(),
        axis.text = element_text( size=17),
        axis.title = element_text( size=21),
        strip.background = element_rect(fill="white"),
        strip.text. = element_text(size = 21, hjust = 0,face ="bold"),
        strip.placement = "inside")+
  guides(size = guide_legend(order=1),
         alpha = guide_legend(order=1));p1


p2=ggplot(data=malla[malla$growth==unique(malla$growth)[2],], 
          aes(x=time, y=x,
              col =organTypes,size=scenario,alpha=scenario))+
  
  geom_line()+
  #scale_color_brewer(palette="Set2")+
  scale_size_manual(breaks=c("drier&warmer","wetter&colder"),name="",
                    values = c(1.5,0.5), drop = FALSE)+ 
  scale_alpha_manual(breaks=c("drier&warmer","wetter&colder"),name="",
                     values = c(0.3,1), drop = FALSE)+ 
  xlab(NULL)+
  facet_grid(ot ~ growth, scales = "free_y")+
  # scale_color_manual(breaks=c("2-1", "3-1", "4-1" ,
  #                             "2-2", "2-3" ,"2-4" ,"3-4"),name="",
  #                    labels=c("r1","stem","leaf"),
  #                    values = c("#8DA0CB","#FC8D62","#66C2A5"))+  
  scale_color_manual(name=NULL,
                     #breaks=c(2,3,4),
                     #labels=c("root","stem","leaf"),
                     values = c("#8DA0CB","#e78ac3","#a6d854","#FC8D62","#66C2A5"))+  
  labs(colour = NULL, linetype =NULL)+
  ylab(NULL)+ 
  #scale_y_continuous(sec.axis = sec_axis(~.*10-7.6, name = "cumulative transpiration"))+
  theme(legend.text = element_text( size=15),
        legend.title = element_blank(),
        legend.position="bottom",#c(0.5,0.1),
        legend.direction = "horizontal",
        #legend.box = "vertical",
        panel.grid.minor = element_line(colour="white"),
        panel.grid.major = element_line(colour="white"),
        panel.background = element_rect(fill="white", color="black"),
        strip.text.y =element_blank(),
        axis.text = element_text( size=17),
        axis.title = element_text( size=21),
        strip.background = element_rect(fill="white"),
        strip.text. = element_text(size = 21, hjust = 0,face ="bold"),
        strip.placement = "inside")+
  guides(size = guide_legend(order=1),
         alpha = guide_legend(order=1));p2
p3 = ggarrange(p1, p2,
               ncol = 2, nrow = 1,
               common.legend = TRUE, legend = "bottom");p3
p4 = annotate_figure(
  p3,
  bottom = text_grob("day of growth (d)", 
                     size = 21),
  left = text_grob("total root length (cm)",
                   rot = 90,size = 21)); p4